# [실습2] LangChain으로 간단한 LLM 챗봇 질의응답 2


## 1. 환경 설정 및 이전 실습
- 실습1에서 진행한 내용을 불러옵니다.

In [1]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama, ChatOpenAI

In [2]:
import os


if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = ""

In [ ]:
# 먼저, gpt-4o-mini 모델을 사용하는 ChatOpenAI 객체를 생성합니다.
llm = ChatOpenAI(model="gpt-4o-mini")

In [4]:
role = "국토교통부 직원"
messages = [
    SystemMessage(f"당신은 {role} 입니다."),
    HumanMessage("당신을 소개해주세요."),
]

response = llm.invoke(messages)

In [ ]:
response

## 2. 챗봇 Chain 구성

조금 전 `llm` object의 반환 값을 확인해보면, 다른 챗봇을 쓸 때 처럼 답변만 출력된 것이 아니라 다양한 메타 데이터 까지 같이 출력된 것을 확인할 수 있습니다.

저희가 ChatGPT를 쓸 때를 생각해보면, 챗봇에 이걸 그대로 출력하는건 좀 부자연스럽습니다.

이를 방지하기 위해, 답변을 parsing하는 `StrOutputParser`를 활용해봅시다.

### 2-1. Output Parser
- ChatOpenAI Agent를 비롯하여 LLM 답변 중 content만 자동으로 추출하는 Tool인 `StrOutputParser`를 사용합니다.

In [6]:
parser = StrOutputParser()

`StrOutputParser`를 사용해봅시다.

In [ ]:
# Parser가 제대로 답변만을 리턴하는지 확인합니다.
parsed_response = parser.invoke(response)
print(parsed_response)

response에서 의도한 대로 텍스트만 추출하는 것을 확인할 수 있습니다.

### 2-2. 간단한 체인 구성

- 저희는 `ChatOpenAI` 를 통해 gpt-4o-mini 모델의 답변을 받았고, 그 받은 답변을 다시 `StrOutputParser`에 입력해서 답변만 추출하였습니다.
- 이 과정을 Chain으로 엮어서 간략화 해봅시다.

In [8]:
# pipe (|) 연산자를 통해 두 객체를 연결해서 하나의 체인으로 만들 수 있습니다.
chain = llm | parser

Chain 역시 "Runnable" 하므로, `invoke` 메서드를 통해 Chain의 각 구성요소의 `invoke` 메서드를 순차적으로 호출할 수 있습니다.

이때 특정 객체의 `invoke` 반환값은 Chain 상에서 연결된 다음 객체의 `invoke` 메서드에 입력됩니다.

In [ ]:
# 체인을 실행하면, 체인에 포함된 모든 객체가 순차적으로 실행되며, 마지막 객체의 결과가 반환됩니다.
# 여기서는 llm 객체가 먼저 실행되고, 그 결과가 parser 객체에 전달됩니다.
chained_response = chain.invoke(messages)
print(chained_response)

별도의 절차 없이 바로 답변만 생성되는 것을 확인할 수 있습니다. 

### 2-3. 프롬프트 템플릿

이제 여러분의 챗봇에 프로그래밍 조수, 시장조사 요원, 그냥 친구 등 다양한 역할을 적용해야 하는 상황이라 가정합시다.

이를 구현할 수 있는 방법은 여러가지가 있지만, 우선 가장 간단한 방법으로 시스템 프롬프트에 '당신은 {역할} 입니다' 를 입력해 보겠습니다.

이 방법이 항상 잘 작동하는 것은 아니지만, 간단한 예시 정도는 구현할 수 있습니다.

사용자의 입력을 받고, 그에 대응하는 답변을 하기 위해서는 사용자의 입력을 적용할 수 있는 프롬프트 템플릿을 적용할 수 있습니다. 

In [10]:
# role에는 "AI 어시스턴트"가, question에는 "당신을 소개해주세요."가 들어갈 수 있습니다.
# Note. 사용한 문자열이 f-string이 아닙니다. 
# 여기서 중괄호로 감싼 텍스트는 LangChain placeholder를 나타내는 문자열입니다
messages_with_variables = [
    ("system", "당신은 {role} 입니다."),
    ("human", "{question}"),
]

In [11]:
prompt = ChatPromptTemplate.from_messages(messages_with_variables)

앞서 저희가 정의했던 코드와 크게 두가지 차이점이 있습니다.
- HumanMessage, SystemMessage 같은게 없고, 튜플에 역할과 프롬프트가 저장되어 있습니다
- 프롬프트에 {question} 같은 placeholder가 있습니다.

### [TODO] pipe를 통해 체인을 구성해보세요.

In [12]:
# pipe (|) 연산자를 통해 여러 객체를 연결해서 하나의 체인으로 만들 수 있습니다.
# 이 경우, prompt 객체를 통해 변수를 적용한 프롬프트가 생성되고, llm 객체를 통해 이 프롬프트를 실행하고, 마지막으로 parser 객체를 통해 결과를 파싱합니다.
chain = ________ | ________ | ________

In [ ]:
chain.invoke({"role": "국토 교통부 직원 김싸피", "question": "당신을 소개해주세요."})

## 3. 챗봇 사용

마지막으로, 여러분이 제작하신 챗봇을 한번 사용해 봅시다.

1. 사용자의 입력을 받아 앞서 정의한 Chain을 실행하고, 그 결과를 반환하는 함수를 정의합니다.

In [14]:
# 간단한 실습이므로 앞서 사용했던 변수를 그대로 함수의 파라미터로 설정했습니다. 
# 다음 실습 부터는 이를 좀 더 고도화 해 볼 것입니다.
def simple_chat(role, question, chain):
    result = chain.invoke({"role": role, "question": question})
    return result

In [ ]:
role = input("제 역할을 입력해주세요: ")
while True:
    question = input("질문을 입력해주세요 (역할을 바꾸고 싶다면 '역할 교체' 를 입력해주세요. 종료를 원하시면 '종료'를 입력해주세요.): ")
    if question == "역할 교체":
        role = input("역할을 입력해주세요: ")
        continue
    elif question == "종료":
        break
    else:
        # chain = prompt | llm | parser
        result = simple_chat(role, question, chain)
        print(result)

대부분의 경우, 입력한 역할에 맞춰 어느 정도 대답하는 것을 확인할 수 있습니다. <br>
현재 챗봇은 다음 한계점이 있습니다.
- 문서나 데이터 기반 추론이 불가능하다.
- Chat History를 기억하지 못한다.

이어지는 실습에서 두 한계를 개선하고, 교통 3대 혁신 전략 문서 기반 QA 봇을 만들어 봅시다.